<h1>Entrenamiento de modelo para clasificar radiografias de pacientes con o sin neumonía</h1>
Este proyecto tiene como objetivo entrenar un modelo de modo que pueda distinguir entre radiografías de pacientes sanos y radiografías de pacientes con neumonía. El modelo de IA se entrena y evalúa utilizando el entorno de Google Colab y datos almacenados en Google Drive. Previo al entrenamiento del modelo, se descomprimen las carpetas con imagenes (se brinda el codigo por si resulta util). Toda la información se trabaja desde Google Drive para facilitar el acceso, también se proporciona el codigo para montar las carpetas correspondientes ubicadas en Google Drive.

<h2>Pasos</h2>

**Carga de Datos:**

Las imágenes de radiografías se clasifican en dos categorías: NORMAL y PNEUMONIA. Previamente, las imagenes comprimidas se descomprimen y ubican en carpetas denominadas 'Train' y 'Test' respectivamente, con el codigo proporcionado.
Las imágenes se cargan desde dichos directorios específicos, segun esten ubicadas en NORMAL o PNEUMONIA se les asignan etiquetas y se preprocesan, ajustándose a un tamaño uniforme y convirtiéndose a escala de grises.
Los datos procesados se guardan en archivos para evitar recargas constantes, esto lo hice en una carpeta nueva denominada imagenesAlmacenadas a modo de evitar realizar la carga constantemente y evitar demoras innecesarias.

**Estructura del Modelo:**

Se utiliza un modelo de red neuronal convolucional (CNN) con varias capas convolucionales y de pooling.
El modelo se compila con el optimizador adam y la función de pérdida sparse_categorical_crossentropy.

**Entrenamiento y Evaluación:**

El modelo se entrena con las imágenes preprocesadas durante 20 épocas.
Se utiliza TensorBoard para el seguimiento del entrenamiento.
La precisión del modelo se evalúa tanto en los datos de entrenamiento como en los datos de prueba.


#Importacion de librerias

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm #barra de progreso
import cv2
from glob import glob

#para redimencionar
import sklearn
import skimage
from skimage.transform import resize

import random
import datetime

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from skimage.color import rgb2gray

print(tf.__version__)


#Montar en el arbol de carpetas contenido en drive (opcional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Listar el contenido de la carpeta principal de mi Drive
!ls /content/drive/My\ Drive/

In [ ]:
cd /content/drive/

#Codigo para descomprimir archivos

In [ ]:
def descomprimir_zip(ruta_zip, directorio_destino):
    # Verificar si el directorio de destino existe, si no, crearlo
    if not os.path.exists(directorio_destino):
        os.makedirs(directorio_destino)

    # Verificar el formato del archivo ZIP
    with open(ruta_zip, 'rb') as archivo:
        firma = archivo.read(4)
        if firma != b'PK\x03\x04':
            raise zipfile.BadZipFile("File is not a zip file")

    # Abrir el archivo ZIP
    with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip:
        # Extraer todo el contenido en el directorio de destino
        archivo_zip.extractall(directorio_destino)

# Ejemplo de uso
ruta_zip = 'carpeta comprimida con imagenes'
directorio_destino = 'destino elegido para descomprimir la carpeta con imagenes'

if os.path.exists(ruta_zip):
    try:
        descomprimir_zip(ruta_zip, directorio_destino)
    except zipfile.BadZipFile as e:
        print(e)
else:
    print("El archivo no existe en la ruta especificada")

#Carga de Datos

In [ ]:
train_dir = "Ruta de carpeta descomprimida con imagenes de entrenamiento"
test_dir =  "Ruta de carpeta descomprimida con imagenes de test"

#Cuando el FLAG se utiliza en false, no se recargan los datos nuevamnete, para evitar demoras innecesarias.
LOAD_FROM_IMAGES = False

#Clasificamos las imagenes añadiendo una etiqueta segun sea imagen normal o con neumonia
def get_data(folder):
    X = []
    Y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['NORMAL']:
                label = 0 #Etiquetas segun donde se ubiquen las imagenes
            elif folderName in ['PNEUMONIA']:
                label = 1 #Etiquetas segun donde se ubiquen las imagenes
            else:
                label = 2 #Etiquetas segun donde se ubiquen las imagenes
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (150, 150, 3),mode='constant',anti_aliasing=True)
                    img_file = rgb2gray(img_file)
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    Y.append(label)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X,Y

#Una vez guardadas las imagenes en los arrays, usamos los arrays con la informacion de test y train, para no cargar constantemente las imagenes
if LOAD_FROM_IMAGES:
    #cargamos las imágenes a los arrays
    X_train, y_train = get_data(train_dir)
    X_test, y_test= get_data(test_dir)

    #grabamos los arrays en archivos
    np.save('Ruta para almacenar imagenes de entrenamiento', X_train) #Cree una carpeta nueva en Google Drive para almacenar toda esta informacion
    np.save('Ruta para almacenar etiquetas de entrenamiento', y_train) #Cree una carpeta nueva en Google Drive para almacenar toda esta informacion
    np.save('Ruta para almacenar imagenes de test', X_test) #Cree una carpeta nueva en Google Drive para almacenar toda esta informacion
    np.save('Ruta para almacenar etiquetas de test', y_test) #Cree una carpeta nueva en Google Drive para almacenar toda esta informacion
else:
    #cargamos los arrays anteriormente grabados para evitar recargas constantes
    X_train = np.load('Ruta para cargar imagenes de entrenamiento')
    y_train = np.load('Ruta para cargar etiquetas de entrenamiento')
    X_test = np.load('Ruta para cargar imagenes de test')
    y_test = np.load('Ruta para cargar etiquetas de test')



#Muestreo

In [ ]:
#impresion de una imagen al azar para visualizar
def plotHistogram(a):
    plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    plt.hist(a.ravel(), bins=255)
    plt.subplot(1, 2, 2)
    plt.imshow(a, cmap='gray', vmin=0, vmax=1)
    plt.show()

plotHistogram(X_train[3])

In [ ]:
print("No Neumonía")
#Encontrar archivos en carpetas y subcarpetas
multipleImages = glob('Ruta de imagenes descomprimidas para entrenamiento sin Neumonia')
i_ = 0
plt.rcParams['figure.figsize'] = (20.0, 20.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128))
    plt.subplot(5, 5, i_+1)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1

In [ ]:
print("Si neumonía")
#Encontrar archivos en carpetas y subcarpetas
multipleImages = glob('Ruta de imagenes descomprimidas para entrenamiento con Neumonia')
i_ = 0
plt.rcParams['figure.figsize'] = (20.0, 20.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128))
    plt.subplot(5, 5, i_+1)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1

In [ ]:
#Comprobacion de balanceo en cantidad de imagenes
plt.figure(figsize=(8,4))
map_characters = {0: 'No Neumonía', 1: 'Si Neumonía'}
dict_characters=map_characters

df = pd.DataFrame()
df["labels"]=y_train
lab = df['labels']
dist = lab.value_counts()
sns.countplot(lab)
print(dict_characters)

#Estructura y Configuracion del modelo

In [ ]:
#Redimension de imagenes acorde a la configuracion del modelo elegido
X_trainReshaped = X_train.reshape(len(X_train),150,150,1)
X_testReshaped = X_test.reshape(len(X_test),150,150,1)

In [ ]:
model = models.Sequential() #Creación de modelo secuencial

#Capas de convolución
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten()) #Conversion de características 2D en un vector de una dimensión
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary() #Muestreo

#Entrenamiento y Evaluación

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="Ruta para almacenar los registros de TensorBoard" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

model.fit(X_trainReshaped,
          y_train,
          epochs=20,
          validation_data = (X_testReshaped,y_test),
          callbacks=[tensorboard_callback])

In [ ]:
test_loss, test_acc = model.evaluate(X_testReshaped, y_test)
print(test_acc)

In [ ]:
test_loss, test_acc = model.evaluate(X_trainReshaped, y_train)